In [1]:
from fc_utils import chat_completion_request, pretty_print_conversation, get_function_call_from_response, process_functions

### Простой запрос к YandexGPT

In [2]:
messages = []
messages.append({"role": "user", "text": "Сколько денег у меня на балансе?"})

response = chat_completion_request(messages)
messages.append(response['result']['alternatives'][0]['message'])

pretty_print_conversation(messages)

user: Сколько денег у меня на балансе?

assistant: К сожалению, я не могу ответить на этот вопрос, так как у меня нет доступа к информации о вашем банковском счёте или балансе. Чтобы узнать, сколько денег у вас на балансе, вам нужно проверить свой банковский счёт через мобильное приложение банка или официальный веб-сайт.



### Использование инструментов

In [3]:
weather_tool = {
            "function": {
                "name": "WeatherTool",
                "description": "Ходит в API и получает погоду в городе",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "Название города, для которого нужно получить погоду."
                        },
                        "units": {
                            "type": "string",
                            "enum": ["metric", "imperial"],
                                "default": "metric",
                                "description": "Единицы измерения температуры. 'metric' для Цельсия, 'imperial' для Фаренгейта."
                            },
                            "regions": {
                                "type": "array",
                                "default": ["center"],
                                "description": "Части города, для которых нужно получить погоду. Например: ['center', 'west', 'east']"
                            },
                            "days": {
                                "type": "integer",
                                "default": 1,
                                "description": "Количество дней, для которых нужно получить погоду, начиная с текущего дня."
                            }
                    },
                    "required": ["city"],
                    "additionalProperties": False
                }
            }
        }

### Спросим о балансе c инструментом balance_tool


In [4]:
balance_tool = {
        "function": {
            "name": "BalanceTool",
            "description": "Получает баланс пользователя в рублях из базы данных.",
            "parameters": {
                "type": "object",
                "properties": {
                    "userId": {
                        "type": "string",
                        "description": "Идентификатор пользователя, для которого нужно получить баланс."
                    }
                },
                "required": ["userId"],
                "additionalProperties": False
            }
        }
    }

In [5]:
session_info = "\n\nИнформация о сессии:\nТекущее время: 12:11. Текущий userId пользователя: wg359g3f39"

messages = []
messages.append({"role": "system", "text": f"Ты - полезный бот, который помогает пользователю с его проблемами. Ты можешь использовать инструменты, чтобы получить актуальную информацию. Но пользоваться ими нужно не всегда. {session_info}"})
messages.append({"role": "user", "text": "Сколько у меня денег на балансе?"})

response = chat_completion_request(messages, tools=[balance_tool])

toolCalls: list = get_function_call_from_response(response)
messages.append({
    'role': 'assistant',
    'toolCallList': {'toolCalls': toolCalls}
})

pretty_print_conversation(messages)

system: Ты - полезный бот, который помогает пользователю с его проблемами. Ты можешь использовать инструменты, чтобы получить актуальную информацию. Но пользоваться ими нужно не всегда. 

Информация о сессии:
Текущее время: 12:11. Текущий userId пользователя: wg359g3f39

user: Сколько у меня денег на балансе?

assistant: {'toolCalls': [{'functionCall': {'name': 'BalanceTool', 'arguments': {'userId': 'wg359g3f39'}}}]}



Выполним функцию, и вернем результат обратно gpt

In [6]:
toolResults: list = process_functions(toolCalls)
messages.append({
    'role': 'assistant',
    'toolResultList': {'toolResults': toolResults}
})

response = chat_completion_request(messages, tools=[balance_tool])
messages.append(response['result']['alternatives'][0]['message'])
pretty_print_conversation(messages)

system: Ты - полезный бот, который помогает пользователю с его проблемами. Ты можешь использовать инструменты, чтобы получить актуальную информацию. Но пользоваться ими нужно не всегда. 

Информация о сессии:
Текущее время: 12:11. Текущий userId пользователя: wg359g3f39

user: Сколько у меня денег на балансе?

assistant: {'toolCalls': [{'functionCall': {'name': 'BalanceTool', 'arguments': {'userId': 'wg359g3f39'}}}]}

assistant: {'toolResults': [{'functionResult': {'name': 'BalanceTool', 'content': "{'userId': 'wg359g3f39', 'balance': 10000}"}}]}

assistant: На вашем балансе 10000 рублей.



### Параллельное использование инструментов

In [17]:
product_list_tool = {
            "function": {
                "name": "ProductListTool",
                "description": "Получает список товаров из базы данных на основе заданных параметров. Позволяет фильтровать товары по диапазону цен, категории и критерию сортировки.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "priceMinMax": {
                            "type": "array",
                            "items": {
                                "type": "number"
                            },
                            "description": "Диапазон цен в формате [минимальная цена, максимальная цена]. Позволяет пользователю указать, какие товары его интересуют в пределах заданного ценового диапазона."
                        },
                        "category": {
                            "type": "string",
                            "description": "Категория товаров, по которой будет осуществляться фильтрация. Например, 'электроника', 'одежда', 'книги', 'дом и сад'. Это позволяет пользователю сузить поиск до определенной группы товаров."
                        },
                        "sortBy": {
                            "type": "string",
                            "enum": ["price", "popularity", "rating"],
                            "description": "Критерий сортировки товаров. Возможные значения: 'price' для сортировки по цене, 'popularity' для сортировки по популярности, 'rating' для сортировки по рейтингу. Это позволяет пользователю получить список товаров в удобном для него порядке."
                        }
                    },
                    "required": ["priceMinMax", "category"],
                    "additionalProperties": False
                }
            }
        }

order_tool = {
        "function": {
            "name": "OrderTool",
            "description": "инструмент, который заказывает товар по его идентификатору productId",
            "parameters": {
                "type": "object",
                "properties": {
                    "productId": {
                        "type": "string",
                        "description": "Идентификатор товара, который нужно заказать."
                    },
                    "quantity": {
                        "type": "integer",
                        "description": "Количество товара для заказа.",
                        "default": 1,
                        "minimum": 1
                    }
                },
                "required": ["productId", "quantity"],
                "additionalProperties": False
            }
        }
    }

In [28]:
session_info = "\n\nИнформация о сессии:\nТекущее время: 12:11. Текущий userId пользователя: wg359g3f39"
tools = [product_list_tool, balance_tool, order_tool]

messages = []
messages.append({"role": "system", "text": f"Ты - полезный бот, который помогает пользователю с его проблемами. Ты можешь использовать инструменты, чтобы получить актуальную информацию. Но пользоваться ими нужно не всегда. Если пользователь ищет товары, то всегда параллельно с вызовом инструмента ProductListTool вызывай и BalanceTool. {session_info}"})
messages.append({"role": "user", "text": "Покажи мне товары категории 'электроника' в диапазоне цен от 1000 до 50000 рублей, отсортированные по популярности, которые я могу себе позволить купить с учетом моего баланса. Показывай только те товары, которые я могу себе позволить купить."})

response = chat_completion_request(messages, tools=tools)

toolCalls: list = get_function_call_from_response(response)
messages.append({
    'role': 'assistant',
    'toolCallList': {'toolCalls': toolCalls}
})

pretty_print_conversation(messages)

system: Ты - полезный бот, который помогает пользователю с его проблемами. Ты можешь использовать инструменты, чтобы получить актуальную информацию. Но пользоваться ими нужно не всегда. Если пользователь ищет товары, то всегда параллельно с вызовом инструмента ProductListTool вызывай и BalanceTool. 

Информация о сессии:
Текущее время: 12:11. Текущий userId пользователя: wg359g3f39

user: Покажи мне товары категории 'электроника' в диапазоне цен от 1000 до 50000 рублей, отсортированные по популярности, которые я могу себе позволить купить с учетом моего баланса. Показывай только те товары, которые я могу себе позволить купить.

assistant: {'toolCalls': [{'functionCall': {'name': 'ProductListTool', 'arguments': {'category': 'электроника', 'priceMinMax': [1000, 50000], 'sortBy': 'popularity'}}}, {'functionCall': {'name': 'BalanceTool', 'arguments': {'userId': 'wg359g3f39'}}}]}



In [29]:
toolResults: list = process_functions(toolCalls)
messages.append({
    'role': 'assistant',
    'toolResultList': {'toolResults': toolResults}
})

response = chat_completion_request(messages, tools=tools)
messages.append(response['result']['alternatives'][0]['message'])
pretty_print_conversation(messages)

system: Ты - полезный бот, который помогает пользователю с его проблемами. Ты можешь использовать инструменты, чтобы получить актуальную информацию. Но пользоваться ими нужно не всегда. Если пользователь ищет товары, то всегда параллельно с вызовом инструмента ProductListTool вызывай и BalanceTool. 

Информация о сессии:
Текущее время: 12:11. Текущий userId пользователя: wg359g3f39

user: Покажи мне товары категории 'электроника' в диапазоне цен от 1000 до 50000 рублей, отсортированные по популярности, которые я могу себе позволить купить с учетом моего баланса. Показывай только те товары, которые я могу себе позволить купить.

assistant: {'toolCalls': [{'functionCall': {'name': 'ProductListTool', 'arguments': {'category': 'электроника', 'priceMinMax': [1000, 50000], 'sortBy': 'popularity'}}}, {'functionCall': {'name': 'BalanceTool', 'arguments': {'userId': 'wg359g3f39'}}}]}

assistant: {'toolResults': [{'functionResult': {'name': 'ProductListTool', 'content': "[{'productId': 'siaomi45

In [30]:
messages.append({"role": "user", "text": 'Как пелось у RX4D в "запрети мне носить сяоми", это лучший микрокомпьютер, топ за свои деньги. Оформи заказ на сяоми 4+'})

response = chat_completion_request(messages, tools=tools)

toolCalls: list = get_function_call_from_response(response)
messages.append({
    'role': 'assistant',
    'toolCallList': {'toolCalls': toolCalls}
})

toolResults: list = process_functions(toolCalls)
messages.append({
    'role': 'assistant',
    'toolResultList': {'toolResults': toolResults}
})

response = chat_completion_request(messages, tools=tools)
messages.append(response['result']['alternatives'][0]['message'])
pretty_print_conversation(messages)

system: Ты - полезный бот, который помогает пользователю с его проблемами. Ты можешь использовать инструменты, чтобы получить актуальную информацию. Но пользоваться ими нужно не всегда. Если пользователь ищет товары, то всегда параллельно с вызовом инструмента ProductListTool вызывай и BalanceTool. 

Информация о сессии:
Текущее время: 12:11. Текущий userId пользователя: wg359g3f39

user: Покажи мне товары категории 'электроника' в диапазоне цен от 1000 до 50000 рублей, отсортированные по популярности, которые я могу себе позволить купить с учетом моего баланса. Показывай только те товары, которые я могу себе позволить купить.

assistant: {'toolCalls': [{'functionCall': {'name': 'ProductListTool', 'arguments': {'category': 'электроника', 'priceMinMax': [1000, 50000], 'sortBy': 'popularity'}}}, {'functionCall': {'name': 'BalanceTool', 'arguments': {'userId': 'wg359g3f39'}}}]}

assistant: {'toolResults': [{'functionResult': {'name': 'ProductListTool', 'content': "[{'productId': 'siaomi45

### Использование инструментов с нестрогими параметрами

In [40]:
products_table_description = """
Таблица "products" предназначена для хранения информации о товарах, доступных в магазине. 
Каждый товар имеет следующие поля:
- productId: уникальный идентификатор товара (TEXT, PRIMARY KEY)
- name: название товара (TEXT, NOT NULL)
- category: категория товара (TEXT, NOT NULL). Например: электроника, одежда, книги, дом, ягоды, сад
- price: цена товара (REAL, NOT NULL)
- num_of_orders: количество заказов данного товара (INTEGER, NOT NULL)
- rating: рейтинг товара (REAL, NOT NULL)
"""

sql_tool = {
        "function": {
            "name": "ask_database",
            "description": f"Use this function to answer user questions about the database. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {products_table_description}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }



In [43]:
session_info = "\n\nИнформация о сессии:\nТекущее время: 12:11. Текущий userId пользователя: wg359g3f39"
tools = [sql_tool, balance_tool, order_tool]

messages = []
messages.append({"role": "system", "text": f"Ты - полезный бот, который помогает пользователю с его проблемами. Ты можешь использовать инструменты, чтобы получить актуальную информацию. Но пользоваться ими нужно не всегда. {session_info}"})
messages.append({"role": "user", "text": "Мне нужны все товары категории 'электроника', отсортированные по рейтингу, ценой меньше 30к."})

response = chat_completion_request(messages, tools=tools)

toolCalls: list = get_function_call_from_response(response)
messages.append({
    'role': 'assistant',
    'toolCallList': {'toolCalls': toolCalls}
})

toolResults: list = process_functions(toolCalls)
messages.append({
    'role': 'assistant',
    'toolResultList': {'toolResults': toolResults}
})

response = chat_completion_request(messages, tools=tools)
messages.append(response['result']['alternatives'][0]['message'])
with open('output.md', 'w', encoding='utf-8') as f:
        assistant_message = response['result']['alternatives'][0]['message']['text']
        f.write(assistant_message)

pretty_print_conversation(messages)

system: Ты - полезный бот, который помогает пользователю с его проблемами. Ты можешь использовать инструменты, чтобы получить актуальную информацию. Но пользоваться ими нужно не всегда. 

Информация о сессии:
Текущее время: 12:11. Текущий userId пользователя: wg359g3f39

user: Мне нужны все товары категории 'электроника', отсортированные по рейтингу, ценой меньше 30к.

assistant: {'toolCalls': [{'functionCall': {'name': 'ask_database', 'arguments': {'query': "SELECT * FROM products WHERE category = 'электроника' AND price < 30000 ORDER BY rating"}}}]}

assistant: {'toolResults': [{'functionResult': {'name': 'ask_database', 'content': "[('3', 'Наушники DEF', 'электроника', 2999.99, 200, 4.2), ('5', 'Планшет JKL', 'электроника', 24999.99, 100, 4.3), ('1', 'Смартфон XYZ', 'электроника', 19999.99, 150, 4.5)]"}}]}

assistant: Вот список всех товаров категории 'электроника', отсортированных по рейтингу, ценой меньше 30к:

1. Смартфон XYZ
   - Название: Смартфон XYZ
   - Категория: электроник

### Запуск функций из Yandex Cloud Serverless Functions

In [2]:
serverless_func_tool = {
        "function": {
            "name": "serverless_func",
            "description": "Запускает функцию на Yandex Cloud Serverless Functions",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Название функции для запуска"
                    }
                },
                "required": ["name"],
            },
        }
    }

In [7]:
tools = [serverless_func_tool]

messages = []
messages.append({"role": "user", "text": "Запусти функцию в yandex cloud, передай имя 'David'"})

response = chat_completion_request(messages, tools=tools)

toolCalls: list = get_function_call_from_response(response)
messages.append({
    'role': 'assistant',
    'toolCallList': {'toolCalls': toolCalls}
})

toolResults: list = process_functions(toolCalls)
messages.append({
    'role': 'assistant',
    'toolResultList': {'toolResults': toolResults}
})

pretty_print_conversation(messages)

user: Запусти функцию в yandex cloud, передай имя 'David'

assistant: {'toolCalls': [{'functionCall': {'name': 'serverless_func', 'arguments': {'name': 'David'}}}]}

assistant: {'toolResults': [{'functionResult': {'name': 'serverless_func', 'content': 'Hello to David from serverless func!'}}]}



### Запрос в Нейро как интсрумент


In [3]:
searchapi_tool = {
        "function": {
            "name": "searchapi",
            "description": "Отправляет запрос в поисковый движок по интернету Нейро и получает ответ",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Запрос для поиска в интернете"
                    }
                },
                "required": ["query"],
            },
        }
    }

In [9]:
tools = [searchapi_tool]

messages = []
messages.append({"role": "system", "text": f"Ты - полезный бот, который помогает пользователю с его проблемами. Старайся по возможности использовать инструмент searchapi, чтобы получить информацию из интернета."})
messages.append({"role": "user", "text": "Погода в Москве"}) # если сделать запрос на "Свежие новости за сегодня", то скорее всего YandexGPT не сможет ответить, так как новости будут содержать острополитические темы, из-за которых запрос улетит в этику

response = chat_completion_request(messages, tools=tools)

toolCalls: list = get_function_call_from_response(response)
messages.append({
    'role': 'assistant',
    'toolCallList': {'toolCalls': toolCalls}
})

toolResults: list = process_functions(toolCalls)
messages.append({
    'role': 'assistant',
    'toolResultList': {'toolResults': toolResults}
})

response = chat_completion_request(messages, tools=tools, model="yandexgpt/rc")
messages.append(response['result']['alternatives'][0]['message'])
pretty_print_conversation(messages)

system: Ты - полезный бот, который помогает пользователю с его проблемами. Старайся по возможности использовать инструмент searchapi, чтобы получить информацию из интернета.

user: Погода в Москве

assistant: {'toolCalls': [{'functionCall': {'name': 'searchapi', 'arguments': {'query': 'Погода в Москве'}}}]}

assistant: {'toolResults': [{'functionResult': {'name': 'searchapi', 'content': 'Ответ SearchAPI:\n**Погода в Москве на 28 ноября 2024 года:**\n* 19:48: температура 0\xa0°C, пасмурно, влажность 89%, ветер 2 м/с, вероятность осадков 0%. [2]\n* 23:00: температура -1\xa0°C, пасмурно, влажность 89%, ветер 2 м/с, вероятность осадков 0%. [2]\n\n**Прогноз на ближайшие дни:**\n* 29 ноября: утро — температура +1\xa0°C, дождь со снегом, влажность 91%, ветер 3 м/с, вероятность осадков 40%. [2] Днём — температура +2\xa0°C, пасмурно, влажность 90%, ветер 3 м/с, вероятность осадков 0%. [2]\n* 30 ноября: утро — температура +1\xa0°C, пасмурно, влажность 89%, ветер 4 м/с, вероятность осадков 0%. [2